In [249]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#### 1. 데이터 로드

In [250]:
df = pd.read_csv('Train.csv')
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [251]:
# dataset 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


#### 2. 목표 및 모델 선정 과정
1) 저지방 제품과 일반 제품 중 경쟁력 있는 품목은 어떤 것인가
    - 판매상품(Item_Type)의 저지방 함량유무
    - 저지방 제품의 최대 소매가의 금액에 따라 판매 금액에 변동이 있는지
    - 저지방이면서 매장점유율이 높을수록, 판매금액의 증가여부 파악하기 위함
    - 매장에 디스플레이 된 상품이 고객에게 많이 노출될수록 판매량이 늘어나는지 파악하기 위함
    - 제품의 가격이 저렴할수록 판매금액이 늘어나는지 파악하기 위함
2) 해결 방법 고안( 1)-1 ) : 단순선형회귀모델을 사용
    - 판매상품으로 groupby하고, ID개수 구하기
        - 판매상품 중 항목의 개수가 많은 상위 8개 항목을 지정하여 필터링
        - 상위 8개 항목의 점유율
    - 매장 점유율 
        - 평균 구하기 또는 0%인 데이터는 삭제 또는 평균값으로 채우기
    - 상품 판매금액
        - 평균 구하기
        - 상품별 금액 차이가 클 것으로 예상하여 데이터 스케일링이 필요할 것으로 보인다.  
    - 매장 점유율의 변동에 따른 상품의 판매금액을 알아보기 위해 단순선형회귀모델을 사용할 것

#### 3. 문제1

- Item_Fat_Content 항목 분류

In [252]:
df.Item_Fat_Content.unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

In [253]:
# Item_Fat_Content를 LF(low fat)과 REG(Regular)로 분류
df['LF'] = df.apply(lambda x: 'LF' if x.Item_Fat_Content == 'Low Fat' else 'REG', axis=1)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,LF
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,LF
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,REG
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,LF
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800,REG
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,LF


- Low Fat 제품 최대판매가, 판매금액

In [254]:
df1 = df[df.LF == 'LF'][['Item_Type', 'Item_MRP', 'Item_Outlet_Sales']]
df1

,Item_Type,Item_MRP,Item_Outlet_Sales
0,Dairy,249.8092,3735.1380
2,Meat,141.6180,2097.2700
4,Household,53.8614,994.7052
7,Snack Foods,107.7622,4022.7636
10,Fruits and Vegetables,45.5402,1516.0266
...,...,...,...
8515,Baking Goods,157.5288,1571.2880
8516,Others,58.7588,858.8820
8518,Snack Foods,214.5218,2778.3834
8520,Health and Hygiene,85.1224,1193.1136


In [255]:
df1 = df1.groupby('Item_Type')[['Item_Outlet_Sales', 'Item_MRP']].mean()
df1

,Item_Outlet_Sales,Item_MRP
Item_Type,,
Baking Goods,1781.871029,121.286145
Breads,2260.105657,138.264948
Breakfast,2507.863738,166.630097
Canned,2231.123365,139.846247
Dairy,2172.820871,148.114798
Frozen Foods,2094.821929,138.064242
Fruits and Vegetables,2181.465002,136.449031
Hard Drinks,2154.073781,137.782089
Health and Hygiene,2018.337403,131.062467


In [256]:
df1.rename(columns={"Item_Outlet_Sales" : "LF_Sales", "Item_MRP" : "LF_MRP"}, inplace=True)
df1.head()

,LF_Sales,LF_MRP
Item_Type,,
Baking Goods,1781.871029,121.286145
Breads,2260.105657,138.264948
Breakfast,2507.863738,166.630097
Canned,2231.123365,139.846247
Dairy,2172.820871,148.114798


- Regular 제품 최대판매가, 판매금액

In [257]:
df2 = df[df.LF == 'REG'][['Item_Type', 'Item_MRP', 'Item_Outlet_Sales']]
df2

,Item_Type,Item_MRP,Item_Outlet_Sales
1,Soft Drinks,48.2692,443.4228
3,Fruits and Vegetables,182.0950,732.3800
5,Baking Goods,51.4008,556.6088
6,Snack Foods,57.6588,343.5528
8,Frozen Foods,96.9726,1076.5986
...,...,...,...
8513,Meat,99.9042,595.2252
8514,Canned,57.5904,468.7232
8517,Frozen Foods,178.8318,3608.6360
8519,Baking Goods,108.1570,549.2850


In [258]:
df2 = df2.groupby('Item_Type')[['Item_Outlet_Sales', 'Item_MRP']].mean()
df2

,Item_Outlet_Sales,Item_MRP
Item_Type,,
Baking Goods,2101.389518,130.800020
Breads,2147.711008,143.661891
Breakfast,1894.257265,128.142575
Canned,2219.638078,139.686583
Dairy,2308.588262,148.988689
Frozen Foods,2170.209006,138.934358
Fruits and Vegetables,2384.678092,151.815404
Hard Drinks,1942.182987,127.736053
Health and Hygiene,1907.175564,127.815185


In [259]:
df2.rename(columns={"Item_Outlet_Sales" : "REG_Sales", "Item_MRP" : "REG_MRP"}, inplace=True)
df2.head()

,REG_Sales,REG_MRP
Item_Type,,
Baking Goods,2101.389518,130.800020
Breads,2147.711008,143.661891
Breakfast,1894.257265,128.142575
Canned,2219.638078,139.686583
Dairy,2308.588262,148.988689


In [263]:
df = df.copy()

In [265]:
df = df1.join(df2)
df

,LF_Sales,LF_MRP,REG_Sales,REG_MRP
Item_Type,,,,
Baking Goods,1781.871029,121.286145,2101.389518,130.800020
Breads,2260.105657,138.264948,2147.711008,143.661891
Breakfast,2507.863738,166.630097,1894.257265,128.142575
Canned,2231.123365,139.846247,2219.638078,139.686583
Dairy,2172.820871,148.114798,2308.588262,148.988689
Frozen Foods,2094.821929,138.064242,2170.209006,138.934358
Fruits and Vegetables,2181.465002,136.449031,2384.678092,151.815404
Hard Drinks,2154.073781,137.782089,1942.182987,127.736053
Health and Hygiene,2018.337403,131.062467,1907.175564,127.815185


In [267]:
df = df.round(2)
df

,LF_Sales,LF_MRP,REG_Sales,REG_MRP
Item_Type,,,,
Baking Goods,1781.87,121.29,2101.39,130.80
Breads,2260.11,138.26,2147.71,143.66
Breakfast,2507.86,166.63,1894.26,128.14
Canned,2231.12,139.85,2219.64,139.69
Dairy,2172.82,148.11,2308.59,148.99
Frozen Foods,2094.82,138.06,2170.21,138.93
Fruits and Vegetables,2181.47,136.45,2384.68,151.82
Hard Drinks,2154.07,137.78,1942.18,127.74
Health and Hygiene,2018.34,131.06,1907.18,127.82


- Low Fat: LF_Sales vs. LF_MRP

In [272]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(df.LF_MRP.values.reshape(-1,1), df.LF_Sales)
lr.score(df.LF_MRP.values.reshape(-1,1), df.LF_Sales)

0.8153788629093068

- Regular: REG_Sales vs. REG_MRP

In [273]:
lr.fit(df.REG_MRP.values.reshape(-1,1), df.REG_Sales)
lr.score(df.REG_MRP.values.reshape(-1,1), df.REG_Sales)

0.8311404486979366